In [ ]:
from bach import DataFrame
import sqlalchemy
import pandas as pd
# from tests.functional.bach.test_data_and_utils import get_bt_with_json_data, get_bt_with_test_data, get_bt_with_food_data

engine = sqlalchemy.create_engine('postgresql://@localhost:5432/postgres')


In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import sys
sys.path.extend([
    '../analysis'
])

from objectiv_bach.util import duplo_basic_features

In [ ]:
from bach.series.series_objectiv import FeatureFrame

In [ ]:
from bach.series.series_objectiv import SeriesGlobalContexts, SeriesLocationStack
from bach.types import _registry
_registry.register_dtype_series(SeriesGlobalContexts, [], override_registered_types=True)
_registry.register_dtype_series(SeriesLocationStack, [], override_registered_types=True)

In [ ]:
basic_features = duplo_basic_features()
full_df = DataFrame.from_model(engine=engine, model=basic_features, index=['event_id'])#[['global_contexts','location_stack']]

In [ ]:
full_df['global_contexts'] = full_df.global_contexts.astype('objectiv_global_context')
full_df['location_stack'] = full_df.location_stack.astype('objectiv_location_stack')

In [ ]:
full_df.feature.head()

In [ ]:
full_df=full_df[full_df.global_contexts.gc.application=='rod-web-demo']

In [ ]:
# full_df=full_df[full_df.global_contexts.gc.application=='objectiv-website']

In [ ]:
feature_frame = FeatureFrame.from_data_frame(bt=full_df, location_stack_column='location_stack', event_column='event_type')

In [ ]:
feature_frame['test_feature'] = feature_frame['location_stack'].json[1:]

In [ ]:
# small data set
feature_frame.count().head()

In [ ]:
full_df_with_created_feature = feature_frame.write_to_full_frame()

In [ ]:
full_df_with_created_feature.count().head()

In [ ]:
# new feature on entire dataset
print(f'rows: {full_df_with_created_feature.count().head()["session_id_count"]}')
full_df_with_created_feature.head()

## do sankey plot only, no interactive graph

In [ ]:
# prepare sankey numbers
feature_frame.stack_flows_from_feature_df()

In [ ]:
feature_frame.display_sankey()

## run dash app in nb

In [ ]:
from jupyter_dash import JupyterDash as Dash

In [ ]:
from sankey_dash import get_app

In [ ]:
app = get_app(Dash, feature_frame)

In [ ]:
app.run_server(mode='inline', height = 1000, port=8053)

In [ ]:
# create features in the sankey, see them here
feature_frame.head()